In [2]:
import numpy as np

In [3]:
X = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
])

y = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1],
    [0, 0, 0]
])

In [148]:
X = np.array([[2.7810836,2.550537003],
              [1.465489372,2.362125076],
              [3.396561688,4.400293529],
              [1.38807019,1.850220317],
              [3.06407232,3.005305973],
              [7.627531214,2.759262235],
              [5.332441248,2.088626775],
              [6.922596716,1.77106367],
              [8.675418651,-0.242068655],
              [7.673756466,3.508563011]])
y = np.array([[0],
     [0],
     [0],
     [0],
     [0],
     [1],
     [1],
     [1],
     [1],
     [1]])

In [201]:
class NNetwork:
    def __init__(self, layers: list, activate_function='sigmoid', cost_function='square_error'):
        self.layers = layers
        self.network = None
        
        if activate_function == 'sigmoid':
            self._activate_function = lambda x: 1/(1 + np.exp(-x))
            self._derivative_activate_function = lambda x: x * (1.0 - x)
            
        if cost_function == 'square_error':
            self._cost_function = lambda x, y: 1/2 * (x - y)**2
            self._derivative_cost_function = lambda x, y: x - y
        
        self._initialize_network()
    
    def _initialize_network(self):
        self.network = list()
        for i in range(1, len(self.layers)):
            layer = np.random.rand(self.layers[i], self.layers[i-1])
            self.network.append({"weights": layer, "forward": None, "deltas": None})
            
    def _forward_propagation(self, inputs: np.array):
        for layer in self.network:
            weigths = layer['weights']
            inputs = self._activate_function(weigths @ inputs)
            layer['forward'] = inputs
        return inputs
            
    def _back_propagation(self, y):
        for index in reversed(range(len(self.network))):
            layer = self.network[index]
            if index == len(self.network)-1:
                layer['deltas'] = (y - layer['forward']) * self._derivative_activate_function(layer['forward'])
            else:
                next_layer = self.network[index+1]
                layer['deltas'] = self._derivative_activate_function(layer['forward']) * (next_layer['deltas'] @ next_layer['weights'])        
    
    def _update_weights(self, inputs, lr):
        for index, layer in enumerate(self.network):
            if index == 0:
                x0, x1 = np.meshgrid(inputs, layer['deltas'])
                dw = lr * (x0 * x1)
            else:
                x0, x1 = np.meshgrid(self.network[index-1]['forward'], layer['deltas'])
                dw = lr * (x0 * x1)
            
            layer['weights'] -= dw
    
    def fit(self, X, y, n_epoch=10000, lr=0.01):
        for i in range(n_epoch):
            for x_, y_ in zip(X, y):
                self._forward_propagation(x_)
                self._back_propagation(y_)
                self._update_weights(x_, lr)

                
    def predict(self, inputs: list):
        return self._forward_propagation(inputs)

In [224]:
nn = NNetwork([2, 3, 3, 1])

In [227]:
nn.fit(X, y, lr=0.00001)

In [228]:
nn.network

[{'weights': array([[0.421644  , 0.79252995],
         [0.50976989, 0.56029118],
         [0.78035381, 0.27347719]]),
  'forward': array([0.9975671 , 0.99720651, 0.99904013]),
  'deltas': array([5.52847002e-06, 1.11827184e-05, 1.80479708e-06])},
 {'weights': array([[0.38952701, 0.91867295, 0.61227664],
         [0.56305197, 0.91977888, 0.19083824],
         [0.26689416, 0.37221805, 0.35890954]]),
  'forward': array([0.87173365, 0.84151538, 0.73026908]),
  'deltas': array([0.00140479, 0.0023051 , 0.00162171])},
 {'weights': array([[0.71234142, 0.97997408, 0.46680236]]),
  'forward': array([0.85650094]),
  'deltas': array([0.01763705])}]